In [1]:
import csv
import math

import numpy as np

### Tensorflow
import tensorflow as tf
from tensorflow.contrib import slim

In [2]:
INPUT_SIZE = 28
SPLIT_SIZE = 4

### train_x = train_x / NORMALIZE
NORMALIZE = 1.0
# NORMALIZE = 255   

In [3]:
### Dataset class
class Dataset(object) :
    
    def __init__(self, path, batch_size, shuffle = True, normalize = 255.0) :
        self.batch_size = batch_size
        self.shuffle    = shuffle
        self.normalize  = normalize
        self.xs, self.ys = self._load_data(path)
        
        self.data_size = self.xs.shape[0]
        self.batch_max = int(math.ceil(self.data_size / float(self.batch_size)))
        
    def _load_data(self, path) :
        reader = csv.reader(open(path), lineterminator = "\n")
        data   = []
        labels = []
        for row in reader :
            tmp = np.asarray([float(x) for x in row])
            
            ### Normalize
            tmp = tmp / self.normalize
            
            data.append(tmp[:INPUT_SIZE*INPUT_SIZE])
            labels.append(tmp[INPUT_SIZE*INPUT_SIZE:])
        
        data   = np.asarray(data)
        labels = np.asarray(labels)
        return (data, labels)

    def __call__(self) :
        indexes = None
        if self.shuffle :
            indexes = np.random.permutation(self.data_size)
        else :
            indexes = np.arange(self.data_size)

        for d_idx in range(self.batch_max) :
            start =     (d_idx + 0) * self.batch_size
            end   = min((d_idx + 1) * self.batch_size, self.data_size)
            
            batch_data_size = len(indexes[start:end])
            xs = np.zeros((batch_data_size, self.xs.shape[1]), dtype = np.float32)
            ys = np.zeros((batch_data_size, self.ys.shape[1]), dtype = np.float32)
            
            for b_idx, r_idx in enumerate(indexes[start:end]) :
                xs[b_idx] = self.xs[r_idx]
                ys[b_idx] = self.ys[r_idx]

            yield (xs, ys)

In [4]:
### Load dataset
dataset = Dataset("training_data_4x4.csv", 10, shuffle = True, normalize = NORMALIZE)

In [5]:
valid_dataset = Dataset("training_data_4x4.csv", 10, shuffle = False, normalize = NORMALIZE)

In [6]:
### Load batch data sample code
# for batch_num, (train_x, train_y) in enumerate(dataset(), 1) :
#     print("Batch {0}".format(batch_num))
#     print("Train shape : {0}".format(train_x.shape))

In [7]:
### for jupyter, interactive session.
sess = tf.InteractiveSession()

In [9]:
### Network settings
NN_INPUT_SIZE  = INPUT_SIZE * INPUT_SIZE
NN_OUTPUT_SIZE = 4 * SPLIT_SIZE

x  = tf.placeholder(tf.float32, shape=[None, NN_INPUT_SIZE])
y_ = tf.placeholder(tf.float32, shape=[None, NN_OUTPUT_SIZE])

def print_shape(layer) :
    print("Layer \"{0}\" shape : {1}".format(layer.name, layer.shape))

with slim.arg_scope([slim.conv2d, slim.fully_connected], activation_fn=tf.nn.relu,
    weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
    biases_initializer=tf.constant_initializer(0.1)):
    with slim.arg_scope([slim.max_pool2d], padding='SAME'):
        x_image = slim.array_ops.reshape(x, [-1, INPUT_SIZE, INPUT_SIZE, 1])
        h_conv1 = slim.conv2d(x_image, 32, [5, 5])
        h_pool1 = slim.max_pool2d(h_conv1, [2, 2])

        h_conv2 = slim.conv2d(h_pool1, 32, [5, 5])
        h_pool2 = slim.max_pool2d(h_conv2, [2, 2])
        
        h_conv3 = slim.conv2d(h_pool2, 64, [5, 5])
        h_pool3 = slim.max_pool2d(h_conv3, [2, 2])

        h_pool3_flat = slim.flatten(h_pool3)
        h_fc1 = slim.fully_connected(h_pool3_flat, 100)

        keep_prob = tf.placeholder(tf.float32)
        h_fc1_drop = slim.dropout(h_fc1, keep_prob)

        y_out  = slim.fully_connected(h_fc1_drop, NN_OUTPUT_SIZE, activation_fn=None)
        
print_shape(h_conv1)
print_shape(h_pool1)
print_shape(h_conv2)
print_shape(h_pool2)
print_shape(h_conv3)
print_shape(h_pool3)
        

Layer "Conv_3/Relu:0" shape : (?, 28, 28, 32)
Layer "MaxPool2D_3/MaxPool:0" shape : (?, 14, 14, 32)
Layer "Conv_4/Relu:0" shape : (?, 14, 14, 32)
Layer "MaxPool2D_4/MaxPool:0" shape : (?, 7, 7, 32)
Layer "Conv_5/Relu:0" shape : (?, 7, 7, 64)
Layer "MaxPool2D_5/MaxPool:0" shape : (?, 4, 4, 64)


In [8]:
### Loss
loss = tf.losses.mean_squared_error(y_, y_out)

In [9]:
### Training step (Optimizer)
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

In [10]:
### Initialize session
sess.run(tf.global_variables_initializer())

In [13]:
### Training
EPOCH = 10000
VALIDATION_EPOCH = 100
for epoch in range(1, EPOCH + 1) :
    
    train_loss = 0
    for train_x, train_y in dataset() :
        _, tmp_loss = sess.run([train_step, loss], feed_dict = {x: train_x, y_: train_y, keep_prob: 0.5})
        batch_loss = tmp_loss / train_x.shape[0]
        train_loss += batch_loss

    ### Validation
    if epoch % VALIDATION_EPOCH == 0 :
        valid_loss = 0
        for (valid_x, valid_y) in valid_dataset() :
            _tmp_loss  = sess.run(loss, feed_dict = {x: train_x, y_: train_y, keep_prob: 1.0})
            batch_loss = _tmp_loss / valid_x.shape[0]
            valid_loss += batch_loss
        print("Epoch {0} train loss = {1}, valid loss = {2}, sqrt(valid loss) * {4} = {3}".format(epoch, train_loss, valid_loss, math.sqrt(valid_loss) * NORMALIZE, NORMALIZE))

Epoch 100 train loss = 1798.553466796875, valid loss = 411.88538818359376, sqrt(valid loss) * 1.0 = 20.29495967435249
Epoch 200 train loss = 1482.091259765625, valid loss = 444.4349304199219, sqrt(valid loss) * 1.0 = 21.08162542167757
Epoch 300 train loss = 1405.4134948730468, valid loss = 224.9304458618164, sqrt(valid loss) * 1.0 = 14.997681349522546
Epoch 400 train loss = 1263.9679748535157, valid loss = 379.33919219970704, sqrt(valid loss) * 1.0 = 19.47663195215505
Epoch 500 train loss = 1474.6268798828125, valid loss = 357.95313720703126, sqrt(valid loss) * 1.0 = 18.919649500110495
Epoch 600 train loss = 1408.1375366210937, valid loss = 319.88295288085936, sqrt(valid loss) * 1.0 = 17.88527195434443
Epoch 700 train loss = 1296.4469360351563, valid loss = 322.1818023681641, sqrt(valid loss) * 1.0 = 17.9494234550351
Epoch 800 train loss = 1398.60234375, valid loss = 306.7316360473633, sqrt(valid loss) * 1.0 = 17.513755623719412
Epoch 900 train loss = 1350.4586181640625, valid loss = 4

Epoch 7000 train loss = 1559.6726928710937, valid loss = 287.37750854492185, sqrt(valid loss) * 1.0 = 16.952212497043618
Epoch 7100 train loss = 1726.553039550781, valid loss = 318.82517395019534, sqrt(valid loss) * 1.0 = 17.85567623894977
Epoch 7200 train loss = 1559.5690185546875, valid loss = 366.10281829833986, sqrt(valid loss) * 1.0 = 19.133813480285102
Epoch 7300 train loss = 1423.204461669922, valid loss = 363.71016998291014, sqrt(valid loss) * 1.0 = 19.07118690545793
Epoch 7400 train loss = 1395.4330810546876, valid loss = 336.85169525146483, sqrt(valid loss) * 1.0 = 18.353519968972297
Epoch 7500 train loss = 1590.6712890625, valid loss = 324.7494415283203, sqrt(valid loss) * 1.0 = 18.02080579575509
Epoch 7600 train loss = 1622.0218994140625, valid loss = 356.3919418334961, sqrt(valid loss) * 1.0 = 18.87834584473693
Epoch 7700 train loss = 1491.4078186035156, valid loss = 295.48864288330077, sqrt(valid loss) * 1.0 = 17.18978309587706
Epoch 7800 train loss = 1474.3798095703123, 

In [31]:
### Save session
saver = tf.train.Saver()
saver.save(sess, "cae_model.ckpt")

'cae_model.ckpt'

In [32]:
### Load session
saver.restore(sess, "cae_model.ckpt")

INFO:tensorflow:Restoring parameters from cae_model.ckpt
